In [1]:
import numpy as np
import pandas as pd

In [2]:
queue=pd.DataFrame(columns=['x','y','z','distancepow2','distance','material'])
#queue=queue.append({'x':0,'y':0,'z':0,'distancepow2':0,'distance':0}, ignore_index=True)
queue

,x,y,z,distancepow2,distance,material


In [3]:
Meterial_spectrum=pd.read_csv('Meterial_spectrum.csv')

In [4]:
Nameid=Meterial_spectrum['Name'].to_dict()

In [5]:
res = dict((v,k) for k,v in Nameid.items())
Nameid=0
Nameid=res
Nameid

{'Bone (Cancellous)': 0,
 'Bone (Cortical)': 1,
 'Bone Marrow (Red)': 2,
 'Bone Marrow (Yellow)': 3,
 'Brain': 4,
 'Brain (Grey Matter)': 5,
 'Brain (White Matter)': 6,
 'Eye (Aqueous Humor)': 7,
 'Eye (Choroid)': 8,
 'Eye (Ciliary Body)': 9,
 'Eye (Cornea)': 10,
 'Eye (Iris)': 11,
 'Eye (Lens)': 12,
 'Eye (Retina)': 13,
 'Eye (Sclera)': 14,
 'Eye (Vitreous Humor)': 15,
 'Eye Lens (Cortex)': 16,
 'Eye Lens (Nucleus)': 17,
 'Muscles': 18,
 'Skin': 19,
 'Skull Cancellous': 20,
 'Skull Cortical': 21,
 'Tooth': 22,
 'Air': 23}

In [6]:
Meterial_id=pd.read_csv('Meterial_id.csv')
Meterial_id['id']='NaN'
for i in range(len(Meterial_id)):
    Meterial_id.ix[i, 'id']=Nameid[Meterial_id.iloc[i].Name]

/Users/mattjunk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [7]:
Meterial_id=Meterial_id.set_index('id')
Meterial_id

,Name,R,G,B
id,,,,
0,Bone (Cancellous),0,0,0
1,Bone (Cortical),0,0,0
2,Bone Marrow (Red),0,0,0
3,Bone Marrow (Yellow),0,0,0
4,Brain,0,0,0
5,Brain (Grey Matter),0,0,0
6,Brain (White Matter),0,0,0
7,Eye (Aqueous Humor),0,0,0
8,Eye (Choroid),0,0,0


In [20]:
(Meterial_id['R']==0) & (Meterial_id['G']==0) & (Meterial_id['B']==0)

id
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
dtype: bool

In [42]:
Meterial_id[(Meterial_id['R']==0) & (Meterial_id['G']==0) & (Meterial_id['B']==0)].index[0]

0

In [43]:
def getid(r=0,g=0,b=0):
    if (len(Meterial_id[(Meterial_id['R']==r) & (Meterial_id['G']==g) & (Meterial_id['B']==b)])):
        print("No record",r,g,b)
        return -1
    else:
        return Meterial_id[(Meterial_id['R']==r) & (Meterial_id['G']==g) & (Meterial_id['B']==b)].index[0]

In [ ]:
from skimage import io
photo=[]
for i in range(1,11):
    img = io.imread("crossec"+str(i)+".jpg")
    photo.append(img)

In [8]:
#z y x
model = np.zeros(shape=(len(photo),photo[0].shape[0],photo[0].shape[1]))
for iz in range(len(photo)):
    print(iz)
    for iy in range(photo[0].shape[0]):
        for ix in range(photo[0].shape[1]):
            queue=queue.append({'x':ix,'y':iy,'z':iz,'distancepow2':0,'distance':0,"material":getid(photo[iz][iy][ix][0],photo[iz][iy][ix][1],photo[iz][iy][ix][2])}, ignore_index=True)

0


KeyboardInterrupt: 

In [ ]:
sounddropx=0
sounddropy=0
sounddropz=0
lengthx=1
lengthy=1
lengthz=1
queue['distancepow2']=((queue['x']-20)*lengthx)**2+((queue['y']-20)*lengthy)**2+((queue['z']-3)*lengthz)**2
queue['distance']=np.sqrt(list(queue['distancepow2']))

In [ ]:
queue = queue.sort_values('distancepow2')
queue=queue.reset_index()

In [9]:
Absobtion=Meterial_spectrum['27.5']

In [46]:
Meterial_spectrum['27.5'][0]

0.0001582

In [47]:
model[sounddropz,sounddropy,sounddropx]=100

co1=1
co2=1
co1z=1
co2z=1
co3z=1
for i in range(1,len(queue)):
    ix=queue.iloc[i].x
    iy=queue.iloc[i].y
    iz=queue.iloc[i].z
    imaterial=queue.iloc[i].material
    Abs=Absobtion[imaterial]#abs per lengthx
    
    val=0
    piece=0
    
    if(ix-1>=0):
        if(model[iz][iy][ix-1]>0):
            piece=piece+co1
            val=val+((model[iz][iy][ix-1])*co1)
        
    if(ix+1<model.shape[2]):
        if(model[iz][iy][ix+1]>0):
            piece=piece+co1
            val=val+((model[iz][iy][ix+1])*co1)
        
    if(iy-1>=0):
        if(model[iz][iy-1][ix]>0):
            piece=piece+co1
            val=val+((model[iz][iy-1][ix])*co1)
        
    if(iy+1<model.shape[1]):
        if(model[iz][iy+1][ix]>0):
            piece=piece+co1
            val=val+((model[iz][iy+1][ix])*co1)
        
    if(iz-1>=0):
        if(model[iz-1][iy][ix]>0):
            piece=piece+co1z
            val=val+((model[iz-1][iy][ix])*co1z)
        
    if(iz+1<model.shape[0]):
        if(model[iz+1][iy][ix]>0):
            piece=piece+co1z
            val=val+((model[iz+1][iy][ix])*co1z)
        
    ####################################
    
    if(ix-1>=0 and iy-1>=0):
        if(model[iz][iy-1][ix-1]>0):
            piece=piece+co2
            val=val+((model[iz][iy-1][ix-1])*co2)
    
    if(ix+1<model.shape[2] and iy-1>=0):
        if (model[iz][iy-1][ix+1]>0):
            piece=piece+co2
            val=val+((model[iz][iy-1][ix+1])*co2)
    
    if(ix-1>=0 and iy+1<model.shape[1]):
        if(model[iz][iy+1][ix-1]>0):
            piece=piece+co2
            val=val+((model[iz][iy+1][ix-1])*co2)
        
    if(ix+1<model.shape[2] and iy+1<model.shape[1]):
        if(model[iz][iy+1][ix+1]>0):
            piece=piece+co2
            val=val+((model[iz][iy+1][ix+1])*co2)
    
    ####
    
    if(ix-1>=0 and iz-1>=0):
        if(model[iz-1][iy][ix-1]>0):
            piece=piece+co2z
            val=val+((model[iz-1][iy][ix-1])*co2z)
        
    if(ix-1>=0 and iz+1<model.shape[0]):
        if(model[iz+1][iy][ix-1]>0):
            piece=piece+co2z
            val=val+((model[iz+1][iy][ix-1])*co2z)
    
    if(ix+1<model.shape[2] and iz-1>=0):
        if(model[iz-1][iy][ix+1]>0):
            piece=piece+co2z
            val=val+((model[iz-1][iy][ix+1])*co2z)
        
    if(ix+1<model.shape[2] and iz+1<model.shape[0]):
        if(model[iz+1][iy][ix+1]>0):
            piece=piece+co2z
            val=val+((model[iz+1][iy][ix+1])*co2z)
    
    ####
    
    if(iy-1>=0 and iz-1>=0):
        if(model[iz-1][iy-1][ix]>0):
            piece=piece+co2z
            val=val+((model[iz-1][iy-1][ix])*co2z)
        
    if(iy-1>=0 and iz+1<model.shape[0]):
        if(model[iz+1][iy-1][ix]>0):
            piece=piece+co2z
            val=val+((model[iz+1][iy-1][ix])*co2z)
    
    if(iy+1<model.shape[1] and iz-1>=0):
        if(model[iz-1][iy+1][ix]>0):
            piece=piece+co2z
            val=val+((model[iz-1][iy+1][ix])*co2z)
        
    if(iy+1<model.shape[1] and iz+1<model.shape[0]):
        if(model[iz+1][iy+1][ix]>0):
            piece=piece+co2z
            val=val+((model[iz+1][iy+1][ix])*co2z)
    
    ####################################
    
    if(ix-1>=0 and iy-1>=0 and iz-1>=0):
        if(model[iz-1][iy-1][ix-1]>0):
            piece=piece+co3z
            val=val+((model[iz-1][iy-1][ix-1])*co3z)
        
    if(ix+1<model.shape[2] and iy+1<model.shape[1] and iz+1<model.shape[0]):
        if(model[iz+1][iy+1][ix+1]>0):
            piece=piece+co3z
            val=val+((model[iz+1][iy+1][ix+1])*co3z)
    
    if(ix-1>=0 and iy-1>=0 and iz+1<model.shape[0]):
        if(model[iz+1][iy-1][ix-1]>0):
            piece=piece+co3z
            val=val+((model[iz+1][iy-1][ix-1])*co3z)
        
    if(ix-1>=0 and iy+1<model.shape[1] and iz-1>=0):
        if(model[iz-1][iy+1][ix-1]>0):
            piece=piece+co3z
            val=val+((model[iz-1][iy+1][ix-1])*co3z)
    
    if(ix-1>=0 and iy+1<model.shape[1] and iz+1<model.shape[0]):
        if(model[iz+1][iy+1][ix-1]>0):
            piece=piece+co3z
            val=val+((model[iz+1][iy+1][ix-1])*co3z)
    
    ## 
    
    if(ix+1<model.shape[2] and iy-1>=0 and iz-1>=0):
        if(model[iz-1][iy-1][ix+1]>0):
            piece=piece+co3z
            val=val+((model[iz-1][iy-1][ix+1])*co3z)
    
    if(ix+1<model.shape[2] and iy-1>=0 and iz+1<model.shape[0]):
        if(model[iz+1][iy-1][ix+1]>0):
            piece=piece+co3z
            val=val+((model[iz+1][iy-1][ix+1])*co3z)
        
    if(ix+1<model.shape[2] and iy+1<model.shape[1] and iz-1>=0):
        if(model[iz-1][iy+1][ix+1]>0):
            piece=piece+co3z
            val=val+((model[iz-1][iy+1][ix+1])*co3z)
            
    model[iz,iy,ix]=(val/piece)*Abs # *sqrt(3,lengthx*lengthy*lengthz)

NameError: name 'model' is not defined

In [ ]:
#q9=queue[queue['z']==3]

In [ ]:
'''
empty9 = np.zeros(shape=(100,100))
for i in range(len(q9)):
    cell=q9.iloc[i]
    empty9[cell.y][cell.x]=cell.distance
'''

In [ ]:
#max(queue['distance'])

In [ ]:
'''
import matplotlib.pyplot as plt
for j in range(10):
    q9=queue[queue['z']==j]
    empty9 = np.zeros(shape=(100,100))
    for i in range(len(q9)):
        cell=q9.iloc[i]
        empty9[cell.y][cell.x]=cell.distance
    plt.imshow(empty9, cmap="RdBu",interpolation='nearest',
               vmin=0, vmax=127)
    plt.show()
'''